In [56]:
import pandas as pd
import numpy as np

# a) Generate a Dataset

*Randomly generate a dataset (dataframe) with eight columns and 50,000 rows. Each column should be a categorical variable (of arbitrary name) with three levels (of arbitrary names) in approximately equal proportions.*

## i. Random Generation

In [57]:
examples=50000
features=8
num_categories=3

#Encode each of the three categories as integers
#Generate random floats from 0-1. Multiply by num_categories to get random numbers over the right range, then round down to get integers.
rand_arr = np.floor(np.random.rand(examples,features)*num_categories).astype(int)
rand_arr

array([[0, 0, 1, ..., 1, 1, 1],
       [0, 2, 1, ..., 1, 1, 0],
       [1, 1, 2, ..., 1, 1, 1],
       ...,
       [1, 0, 2, ..., 1, 0, 1],
       [1, 2, 2, ..., 0, 2, 1],
       [0, 1, 0, ..., 0, 2, 2]])

## ii. Convert to a Pandas Dataframe

In [58]:
df = pd.DataFrame(rand_arr)

#Rename the columns for clarity
col_numbers = range(0,8)
col_names = [f"Column {col}" for col in col_numbers]
#generate a dictionary of column numbers and names to pass to the dataframe rename method
columns_rename = {number:name for (number,name) in zip(col_numbers, col_names)}
df.rename(columns=columns_rename, inplace=True)

df


,Column 0,Column 1,Column 2,Column 3,Column 4,Column 5,Column 6,Column 7
0,0,0,1,1,1,1,1,1
1,0,2,1,1,2,1,1,0
2,1,1,2,0,0,1,1,1
3,1,0,0,1,1,2,2,1
4,1,1,1,1,1,1,1,2
...,...,...,...,...,...,...,...,...
49995,2,1,1,2,2,2,2,0
49996,2,2,0,0,0,1,1,2
49997,1,0,2,1,1,1,0,1
49998,1,2,2,1,2,0,2,1


# b) Verify Distribution

*Verify that the proportions of each value are similar for each of the eight columns.*

In [59]:
df_counts = pd.DataFrame()

#for each of the integer encoded labels, create a series with the number of instances of that label for each column and append that to a new dataframe
for val in range(0,num_categories):
    df_counts = df_counts.append(df[df == val].count(), ignore_index=True)

df_counts.index.set_names('Category Label', inplace=True)
print("Count of Category Labels by Column:")
df_counts

Count of Category Labels by Column:


,Column 0,Column 1,Column 2,Column 3,Column 4,Column 5,Column 6,Column 7
Category Label,,,,,,,,
0,16933.0,16582.0,16563.0,16591.0,16693.0,16692.0,16379.0,16580.0
1,16419.0,16712.0,16618.0,16746.0,16642.0,16576.0,16861.0,16550.0
2,16648.0,16706.0,16819.0,16663.0,16665.0,16732.0,16760.0,16870.0


The proportions of each label are are similar for each column: within expectations of what we would see from random generation.

# C) Unique Combinations

*How many unique rows (i.e., permutations of category levels) are possible?*

In [66]:
possible_permutations = num_categories**features
print("There are {:,} unique permutations.".format(possible_permutations))

There are 6,561 unique permutations.


# d) Permutation Frequency

*Produce a table and appropriate graph which show the frequencies (numbers of groups) by permutation group sizes up to group size of 12. That is, how many groups are unique combinations (group size = 1), how many groups are made up of a pair of matching combinations (group size = 2), how many groups are made up three the same, etc?*


## i) Create Table

Count the number of instances of each permutation:

In [61]:
s_perm_frequency = df.pivot_table(index=col_names, aggfunc='size').rename("Count")
s_perm_frequency


Column 0  Column 1  Column 2  Column 3  Column 4  Column 5  Column 6  Column 7
0         0         0         0         0         0         0         0            6
                                                                      1            4
                                                                      2            6
                                                            1         0            8
                                                                      1            4
                                                                                  ..
2         2         2         2         2         2         1         1            5
                                                                      2           14
                                                            2         0            9
                                                                      1            6
                                                                      2

Group by the permutation count and count the size of the groups:

In [62]:
#count
s_permutation_count = s_perm_frequency.value_counts().sort_index()
#convert to dataframe
df_permutation_count = s_permutation_count.to_frame()
#name the index
df_permutation_count.index.set_names("Group Size", inplace=True)
# Cut off after group size of 12
max_group_size = 12
df_permutation_count = df_permutation_count.loc[df_permutation_count.index <= max_group_size]
df_permutation_count

,Count
Group Size,
1,20
2,83
3,212
4,451
5,711
6,937
7,927
8,924
9,741


## ii) Plot

In [63]:
import plotly.express as px

px.line(df_permutation_count.reset_index(), x = "Group Size", y = "Count", title="Unique Permuations")

# e) Distribution

*Comment Upon the Distribution of Group Sizes in d)*

The group sizes appear to be normally distributed, which is what would be expected from randomly generated data.

# f) Privacy Implications

*If your random variables were, in fact, meaningful information on individuals, which group sizes are of most concern from a privacy perspective?*

Assuming that the individual category labels are not personally identifying, privacy would become a concern when combinations of these anonymous pieces of information become identifying. Therefore it is the small group sizes in the above dataset that would be of concern.

For example, if this was an equality monitoring survey (which should be completely anonymous) conducted in a work place: if we had some prior information on the employees, we may be able to identify the survey responses of a target individual and have access to what should be anonymous information about them. This is clearly undesirable from a privacy perspective. The smaller the group size, the fewer pieces of prior information we need to identify individuals. 

# g) Missing Data

*Consider the effect of missing data in the dataset you created in Part a).  How might this complicate the production of a frequency table of group sizes in Part d)?*

If we ignore missing data (pandas would do this by default in the value_counts method that produces the frequency table), this would have the effect of shifting the frequency graph to the left and reducing the average group size: the data would become less anonymous. 

However, that is probably the wrong approach. For anonymity purposes, we should treat missing data as an extra category label. This increases the number of possible permutations, the average group size would decrease and the data would be less anonymous. 

The change would depend on how much data is missing, but in our current example, if the amount of missing data is weighted as equally as our 3 labels across all 8 columns, the number of possible permutations would increase by a factor of 8 and have a dramatic effect on the group size and level of anonymity. 

# h) Deployment

*Imagine the code that you wrote for Part d) was to be deployed in an automated system that Mirador’s customers could use independently, on potentially large volumes of their own data. Describe how you might deploy the code, and what additional considerations you might have or any changes to the code you might make.*

- Deployment:
    - Cloud compute such as AWS EC2
    - Cloud storage such as AWS RDS or dynamoDB

- Other considerations:
    - Would need to consider the approach to missing data in respect to g)

- Changes to code:
    - Code is already flexible: can account for any number of columns, examples, or categories
    - Migrate from jupyter notebook to python file, turn code segments into a class/classes and create an importable python module.
    - Would need to add an integer encoding step for categorical labels for real data
    - Group size cutoff: currently manually specified, could maybe be calculated from the distribution to account for different datasets (e.g. mean +2 standard deviations).



